# Fáza 2 - Predspracovanie údajov

__Autori:__ Dávid Penťa, Samuel Bernát
__Percentuálny podiel práce:__ 50% / 50%

V tejto fáze sa od Vás očakáva že realizujte predspracovanie údajov pre strojové učenie. Výsledkom bude upravená dátová sada (csv alebo tsv), kde jedno pozorovanie je opísané jedným riadkom.
- scikit-learn vie len numerické dáta, takže treba niečo spraviť s nenumerickými dátami.
- Replikovateľnosť predspracovania na trénovacej a testovacej množine dát, aby ste mohli
zopakovať predspracovanie viackrát podľa Vašej potreby (iteratívne).

Keď sa predspracovaním mohol zmeniť tvar a charakteristiky dát, je možné že treba realizovať EDA opakovane podľa Vašej potreby. Bodovanie znovu (EDA) nebudeme, zmeny ale dokumentujte. Problém s dátami môžete riešiť iteratívne v každej fáze aj vo všetkých fázach podľa potreby.

In [226]:
import statsmodels.api as sm
import dateparser as dateparser
import matplotlib
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats
import pylab as py
import statsmodels.stats.api as sms
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy.stats import pearsonr
import math

In [227]:
measurements_file = "data/measurements.csv"
measurements_data = pd.read_csv(measurements_file, sep='\t')

stations_file = "data/stations.csv"
stations_data = pd.read_csv(stations_file, sep='\t')

## Integrácia a čistenie dát
Transformujte dáta na vhodný formát pre strojové učenie t.j. jedno pozorovanie musí byť opísané jedným riadkom a každý atribút musí byť v numerickom formáte.
- Pri riešení chýbajúcich hodnôt (missing values) vyskúšajte rôzne stratégie ako napr.
    - odstránenie pozorovaní s chýbajúcimi údajmi
    - nahradenie chýbajúcej hodnoty mediánom, priemerom, pomerom (ku korelovanému atribútu), alebo pomocou lineárnej regresie resp. kNN
- Podobne postupujte aj pri riešení vychýlených hodnôt (outlier detection):
    - odstránenie vychýlených (odľahlých) pozorovaní
    - nahradenie vychýlenej hodnoty hraničnými hodnotami rozdelenia (5% resp. 95%)

In [228]:
measurements_data.dropna(inplace=True)
stations_data.dropna(inplace=True)

stations_data["QoS"] = np.where(stations_data["QoS"] == "accep", "acceptable", stations_data["QoS"])
stations_data["QoS"] = np.where(stations_data["QoS"] == "maitennce", "maintenance", stations_data["QoS"])

# stations_data['revision'] = stations_data['revision'].apply(lambda x: pd.Timestamp(x).strftime('%B-%d-%Y'))
# stations_data['revision_timestamp'] = stations_data['revision'].apply(lambda x: pd.Timestamp(x).timestamp())
stations_data['revision'] = stations_data['revision'].apply(lambda x: pd.Timestamp(x).timestamp())

stations_data['latitude'] = stations_data['latitude'].round(5)
stations_data['longitude'] = stations_data['longitude'].round(5)

stations_data["station"] = np.where(stations_data["station"] == "T‚Äôaebaek", "Taebaek", stations_data["station"])
stations_data["station"] = np.where(stations_data["station"] == "'Ali Sabieh", "Ali Sabieh", stations_data["station"])
stations_data["station"] = np.where(stations_data["station"] == "Oktyabr‚Äôskiy", "Oktyabrsk", stations_data["station"])
stations_data["station"] = np.where(stations_data["station"] == "Roslavl‚Äô", "Roslavl", stations_data["station"])
stations_data["station"] = np.where(stations_data["station"] == "Dyat‚Äôkovo", "Dyatkovo", stations_data["station"])

stations_data

,QoS,station,code,latitude,longitude,revision
0,good,Casa Blanca,MX,19.04222,-98.11889,1.574813e+09
1,building,Mikhaylovka,RU,50.06000,43.23790,1.501027e+09
2,building,Shahre Jadide Andisheh,IR,35.68030,51.01930,1.370045e+09
3,building,Aracaju,BR,-10.91111,-37.07167,1.356739e+09
4,maintenance,Parola,IN,20.88098,75.11937,1.562285e+09
...,...,...,...,...,...,...
1105,building,Tadmur,SY,34.56240,38.28402,1.373414e+09
1106,average,Jizzax,UZ,40.11583,67.84222,1.369872e+09
1107,acceptable,West Chester,US,39.96097,-75.60804,1.351987e+09
1108,acceptable,Oktyabrsk,RU,54.48147,53.47103,1.429315e+09


In [229]:
stations_data = stations_data.groupby(by='station').apply(lambda x: x.loc[x['revision']==x['revision'].max()])

## Spojenie tabuliek

In [230]:
df=pd.merge(stations_data, measurements_data, on=['latitude', 'longitude'], how='outer')

df

,QoS,station,code,latitude,longitude,revision,PM10,CO,Pb,C2H3NO5,...,O3,TEMP,NOx,SO2,NH3,CH4,PRES,PM2.5,warning,PAHs
0,maintenance,Aberdeen,GB,57.14369,-2.09814,1.545523e+09,8.94040,8.15390,51.70980,1.07014,...,7.59161,6.60825,7.42432,8.52814,10.37150,8.04118,1113.51031,8.45419,0.0,7.47770
1,maintenance,Aberdeen,GB,57.14369,-2.09814,1.545523e+09,9.92679,7.48092,50.20703,2.64911,...,7.63081,6.16413,8.41506,8.10514,8.64142,5.75635,1141.89103,8.98874,0.0,5.31318
2,maintenance,Aberdeen,GB,57.14369,-2.09814,1.545523e+09,7.70897,7.94669,30.48851,0.46590,...,9.90479,7.45631,6.88163,8.67120,7.16353,8.02152,1141.32144,8.99272,0.0,5.60567
3,maintenance,Aberdeen,GB,57.14369,-2.09814,1.545523e+09,8.79345,7.95665,41.96855,0.91801,...,7.43302,11.97880,9.63301,8.68320,8.31095,8.63908,1051.38525,10.07743,0.0,6.80804
4,maintenance,Aberdeen,GB,57.14369,-2.09814,1.545523e+09,8.19808,7.87163,25.72025,0.59423,...,5.30181,2.99136,8.13061,9.95595,5.97990,6.37283,1153.65647,9.25494,0.0,7.14369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11349,NaN,NaN,NaN,-21.98333,16.91667,NaN,6.41131,9.07535,47.70567,0.12499,...,8.58258,7.08411,8.29453,8.80690,9.12335,6.29809,1121.92686,7.88076,0.0,4.36186
11350,NaN,NaN,NaN,-21.98333,16.91667,NaN,8.18884,7.50654,47.30981,0.55261,...,10.25785,2.47872,9.02017,6.69832,8.54238,7.32523,1117.36876,10.47664,1.0,7.76151
11351,NaN,NaN,NaN,-21.98333,16.91667,NaN,7.12199,7.28879,35.91866,0.38149,...,5.69174,24.94675,8.63516,8.07599,5.86493,5.76701,1126.98373,9.05705,1.0,9.44614
11352,NaN,NaN,NaN,-21.98333,16.91667,NaN,7.28461,9.27219,36.39219,0.39654,...,4.81725,26.58845,9.33447,7.66379,4.88803,6.04203,1181.85027,7.79083,1.0,7.77353


### Opis každého atribútu numerickým formátom

In [231]:
df['QoS_ID'] = -1

df.loc[df.QoS == "excellent", "QoS_ID"] = "1"
df.loc[df.QoS == "good", "QoS_ID"] = "2"
df.loc[df.QoS == "average", "QoS_ID"] = "3"
df.loc[df.QoS == "acceptable", "QoS_ID"] = "4"
df.loc[df.QoS == "building", "QoS_ID"] = "5"
df.loc[df.QoS == "maintenance", "QoS_ID"] = "5"

df[['QoS_ID']] = df[['QoS_ID']].apply(pd.to_numeric)

In [232]:
df['station_ID'] = df['station'].rank(method='dense')
df['code_ID'] = df['code'].rank(method='dense')

df

,QoS,station,code,latitude,longitude,revision,PM10,CO,Pb,C2H3NO5,...,SO2,NH3,CH4,PRES,PM2.5,warning,PAHs,QoS_ID,station_ID,code_ID
0,maintenance,Aberdeen,GB,57.14369,-2.09814,1.545523e+09,8.94040,8.15390,51.70980,1.07014,...,8.52814,10.37150,8.04118,1113.51031,8.45419,0.0,7.47770,5,1.0,38.0
1,maintenance,Aberdeen,GB,57.14369,-2.09814,1.545523e+09,9.92679,7.48092,50.20703,2.64911,...,8.10514,8.64142,5.75635,1141.89103,8.98874,0.0,5.31318,5,1.0,38.0
2,maintenance,Aberdeen,GB,57.14369,-2.09814,1.545523e+09,7.70897,7.94669,30.48851,0.46590,...,8.67120,7.16353,8.02152,1141.32144,8.99272,0.0,5.60567,5,1.0,38.0
3,maintenance,Aberdeen,GB,57.14369,-2.09814,1.545523e+09,8.79345,7.95665,41.96855,0.91801,...,8.68320,8.31095,8.63908,1051.38525,10.07743,0.0,6.80804,5,1.0,38.0
4,maintenance,Aberdeen,GB,57.14369,-2.09814,1.545523e+09,8.19808,7.87163,25.72025,0.59423,...,9.95595,5.97990,6.37283,1153.65647,9.25494,0.0,7.14369,5,1.0,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11349,NaN,NaN,NaN,-21.98333,16.91667,NaN,6.41131,9.07535,47.70567,0.12499,...,8.80690,9.12335,6.29809,1121.92686,7.88076,0.0,4.36186,-1,NaN,NaN
11350,NaN,NaN,NaN,-21.98333,16.91667,NaN,8.18884,7.50654,47.30981,0.55261,...,6.69832,8.54238,7.32523,1117.36876,10.47664,1.0,7.76151,-1,NaN,NaN
11351,NaN,NaN,NaN,-21.98333,16.91667,NaN,7.12199,7.28879,35.91866,0.38149,...,8.07599,5.86493,5.76701,1126.98373,9.05705,1.0,9.44614,-1,NaN,NaN
11352,NaN,NaN,NaN,-21.98333,16.91667,NaN,7.28461,9.27219,36.39219,0.39654,...,7.66379,4.88803,6.04203,1181.85027,7.79083,1.0,7.77353,-1,NaN,NaN


### Uloženie upravenej dátovej sady do .csv súboru

In [233]:
df[['station_ID', 'code_ID', 'QoS_ID', 'warning', 'latitude', 'longitude', 'revision','PAHs', 'PM10', 'CO', 'Pb', 'C2H3NO5', 'CFCs', 'H2CO', 'O3', 'TEMP', 'NOx', 'SO2', 'NH3', 'CH4', 'PRES', 'PM2.5']].to_csv('output.csv', index=False)